In [35]:
import mesa
from enum import Enum


In [72]:
# Cleaning agent cleans the dirt agent using wealth variable.
# When dirt agent is cleaned it will change its dirt to 0.
# Agents do not collide.
# Cleaning agents move randomly.
Direction = Enum('Direction', ['UP', 'DOWN', 'LEFT', 'RIGHT','NONE'])
AgentType = Enum('AgentType', ['ROAD', 'CAR', 'TS','TR','TL', 'X'])

def right(direction, pos):
    if direction == Direction.UP:
        return pos.x +1, pos.y+1
    
    if direction == Direction.DOWN:
        return pos.x -1, pos.y-1

    if direction == Direction.LEFT:
        return pos.x -1, pos.y+1

    if direction == Direction.RIGHT:
        return pos.x +1, pos.y-1


def left(direction, pos):
    if direction == Direction.UP:
        return pos.x -1, pos.y+1
    
    if direction == Direction.DOWN:
        return pos.x +1, pos.y-1

    if direction == Direction.LEFT:
        return pos.x -1, pos.y-1

    if direction == Direction.RIGHT:
        return pos.x +1, pos.y+1

def front(direction, pos):
    if direction == Direction.UP:
        return pos.x, pos.y+2
    
    if direction == Direction.DOWN:
        return pos.x, pos.y-2

    if direction == Direction.LEFT:
        return pos.x -2, pos.y

    if direction == Direction.RIGHT:
        return pos.x +2, pos.y
    
class RoadAgent(mesa.Agent):

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.AgentType = AgentType.ROAD
    def step(self):
        pass
    def advance(self):
        pass
class tJunctionSAgent(mesa.Agent):
    pass

class tJunctionRAgent(mesa.Agent):
    pass

class tJunctionLAgent(mesa.Agent):
    pass

class fourWayAgent(mesa.Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.greenLeft = False
        self.greenUp = True
    def step(self):
        counter += 1
    def advance(self):
        if counter == 5:
            greenLeft = !greenLeft
            greenUp = !greenUp
            counter = 0
    
            
class CarAgent(mesa.Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.direction = Direction.NONE
        self.desiredPos = (0,0)
        self.nextPos = (0,0)
        self.jPos = (0,0)
    def move(self):
        if self.direction == Direction.UP:
            self.desiredPos = (self.pos[0],self.pos[1]+1)
        if self.direction == Direction.DOWN:
            self.desiredPos = (self.pos[0],self.pos[1]-1)

        if self.direction == Direction.LEFT:
            self.desiredPos = (self.pos[0]-1,self.pos[1])

        if self.direction == Direction.RIGHT:
             self.desiredPos = (self.pos[0]+1,self.pos[1])

    def step(self):
        move(self)
        contents = self.model.gridRoad.get_cell_list_contents([self.desiredPos])
        if len(contents) > 1:
            self.nextPos = self.pos
        elif isinstance(contents[0],fourWayAgent):
            self.jPos = right(self.direction,self.pos)
            if contents[0].greenUp and self.direction == Direction.UP:
                self.nextPos = self.desiredPos
                self.direction = Direction.RIGHT
    def advance(self):
        self.model.gridRoad.move_agent(self, new_position)
        

In [73]:
myMap = [[0, 1, 0, 0, 0, 0],
           [0, 1, 0, 0, 0, 0],
           [1, 4, 1, 1, 1, 1],
           [0, 1, 0, 0, 0, 0]]

In [77]:
class RoadModel(mesa.Model):
    def __init__(self, roadMap):
        #self.num_agents = numberOfCars
        self.gridRoad = mesa.space.MultiGrid(len(roadMap), len(roadMap[0]), False)
        self.schedule = mesa.time.SimultaneousActivation(self)
        self.counter = 0
        
        # Create agents
        for i in range(len(roadMap)):
            for j in range(len(roadMap[i])):
                if roadMap[i][j] == 1:
                    agent = RoadAgent(self.counter, self)
                    self.schedule.add(agent)
                    self.counter +=1
                    self.gridRoad.place_agent(agent,(i,j))
                if roadMap[i][j] == 4:
                    agent = fourWayAgent(self.counter, self)
                    self.schedule.add(agent)
                    self.counter +=1
                    self.gridRoad.place_agent(agent,(i,j))                    
        agent = CarAgent(self.counter, self)
        agent.direction = Direction.UP
        self.schedule.add(agent)
        self.counter +=1
        self.gridRoad.place_agent(agent,(3,1))
        agent = CarAgent(self.counter, self)
        agent.direction = Direction.RIGHT
        self.schedule.add(agent)
        self.counter +=1
        self.gridRoad.place_agent(agent,(2,0))
    def step(self):
        self.schedule.step()

In [78]:
print(myMap[0][1])

1


In [83]:
import mesa


def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "Color": "red",
                 "r": 0.5}
    return portrayal

grid = mesa.visualization.CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = mesa.visualization.ModularServer(RoadModel, [grid], "Money Model", {"roadMap": myMap})

server.port = 8521 # The default
server.launch()

Interface starting at http://127.0.0.1:8521


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
Socket opened!
{"type":"reset"}
